In [6]:
# Welchen Anteil am Gesamtserlo Content nimmt Real/Mittelschule ggü. Gymansium ein?
# Und gibt es weitere Kategorisierungen, nach denen wir unseren Content analysieren können?

import mysql.connector
from IPython.display import display, Markdown

connection = mysql.connector.connect(
    user="root", password="secret", host="localhost", database="serlo"
)

c = connection.cursor()

def queryone(sql, parameters=[]):
    c.execute(sql, parameters)
    return c.fetchone()


def queryall(sql, parameters=[]):
    c.execute(sql, parameters)
    return c.fetchall()


def get_all_taxonomy_children(taxonomies, children=[]):
    for taxonomy in taxonomies:
        queried_children = queryall(
            """
           SELECT term_taxonomy.id FROM term_taxonomy
           JOIN uuid ON uuid.id = term_taxonomy.id
           JOIN uuid parent ON parent.id = parent_id
           WHERE parent_id = %s
               AND uuid.trashed = 0
               AND parent.trashed = 0
        """,
            [taxonomy],
        )
        queried_children = [child_row[0] for child_row in queried_children]
        children += queried_children
        if len(queried_children) > 0:
            children = get_all_taxonomy_children(queried_children, children)
    return children


def get_all_entities_from_taxonomy(taxonomy):
    entities = queryall(
        """
        SELECT entity_id FROM term_taxonomy_entity
        JOIN uuid ON uuid.id = entity_id
        WHERE term_taxonomy_id = %s
            AND trashed = 0
    """,
        [taxonomy],
    )
    entities = [entity_row[0] for entity_row in entities]
    for entity in entities:
        type_name = queryone(
            """
            SELECT type.name FROM entity 
             JOIN type ON entity.type_id = type.id
             WHERE entity.id = %s
        """,
            [entity],
        )
        if type_name[0] == "course":
            entity_children = queryall(
                """
             SELECT child_id FROM entity_link 
             WHERE parent_id = %s
            """,
                [entity],
            )
            entity_children = [entity_row[0] for entity_row in entity_children]
            entities += entity_children
    return entities


def get_all_entities_from_taxonomy_tree(taxonomies):
    all_taxonomies = list(set(get_all_taxonomy_children(taxonomies)))
    all_entities = []
    for taxonomy in all_taxonomies:
        all_entities += get_all_entities_from_taxonomy(taxonomy)
    all_entities = list(set(all_entities))
    return all_entities

In [2]:
total_content = queryone(
    f"""
    SELECT COUNT(*) AS n FROM entity
        JOIN uuid ON uuid.id = entity.id
        WHERE trashed = 0
    """
)[0]

total_german = queryone(
    f"""
    SELECT COUNT(*) AS n FROM entity
        JOIN uuid ON uuid.id = entity.id
        WHERE trashed = 0
            AND instance_id = 1 
    """
)[0]

MATHE = 5
all_math_entities = get_all_entities_from_taxonomy_tree([MATHE])

display(
    Markdown(
        f"""
### Wir haben insgesamt {total_content} Inhalte, die auf der Platform live sind.
### {total_german} sind in deutscher Version
### Darunter sind {len(all_math_entities)} im Bereich Mathematik

Anm.: Als Inhalt verstehen wir nur Entitäten (z.B. Artikel, Kurse), keine Taxonomien (z.B. Aufgabenordner).

### Limitierungen
- Wir werden uns nur auf Mathe beschränken, weil es der einzige Bereich auf serlo.org,
der nach Schultyp kategorisiert hat.
- Wir basieren uns auf [https://de.serlo.org/mathe/16030/deutschland](https://de.serlo.org/mathe/16030/deutschland).
Es gibt natürlich andere Inhalte, die für bestimmte Schultypen geeignet wären, nur können wir sie nicht einfach
kategorisieren.
"""
    )
)


### Wir haben insgesamt 18187 Inhalte, die auf der Platform live sind.
### 15749 sind in deutscher Version
### Darunter sind 9266 im Bereich Mathematik

Anm.: Als Inhalt verstehen wir nur Entitäten (z.B. Artikel, Kurse), keine Taxonomien (z.B. Aufgabenordner).

### Limitierungen
- Wir werden uns nur auf Mathe beschränken, weil es der einzige Bereich auf serlo.org,
der nach Schultyp kategorisiert hat.
- Wir basieren uns auf [https://de.serlo.org/mathe/16030/deutschland](https://de.serlo.org/mathe/16030/deutschland).
Es gibt natürlich andere Inhalte, die für bestimmte Schultypen geeignet wären, nur können wir sie nicht einfach
kategorisieren.


In [3]:
first_level_gymnasium_taxonomies = [
    181782,
    16042,
    95753,
    172042,
    97947,
    307330,
    34199,
    34730,
]

display(
    Markdown(
        f"""
### Die folgenden Ordner werden als geeignet für Gymnasium betrachtet:
"""
    )
)
for g in first_level_gymnasium_taxonomies:
    display(Markdown(f"[https://serlo.org/{g}](https://serlo.org/{g})"))

first_level_realmittel_taxonomies = [
    223308,
    16259,
    16359,
    16157,
    305819,
    305843,
    95751,
    97943,
    97944,
    97945,
    97946,
    305760,
    34201,
]

display(
    Markdown(
        f"""
### Die folgenden Ordner werden als geeignet für Real-, Mittel- oder Gesamtschule betrachtet:
"""
    )
)
for r in first_level_realmittel_taxonomies:
    display(Markdown(f"[https://serlo.org/{r}](https://serlo.org/{r})"))

display(Markdown("Diese Ordner werden jetzt durchgesucht."))


### Die folgenden Ordner werden als geeignet für Gymnasium betrachtet:


[https://serlo.org/181782](https://serlo.org/181782)

[https://serlo.org/16042](https://serlo.org/16042)

[https://serlo.org/95753](https://serlo.org/95753)

[https://serlo.org/172042](https://serlo.org/172042)

...

[https://serlo.org/97945](https://serlo.org/97945)

[https://serlo.org/97946](https://serlo.org/97946)

[https://serlo.org/305760](https://serlo.org/305760)

[https://serlo.org/34201](https://serlo.org/34201)

Diese Ordner werden jetzt durchgesucht.

In [5]:
all_gymnasium_entities = get_all_entities_from_taxonomy_tree(
    first_level_gymnasium_taxonomies
)

In [7]:
all_realmittel_entities = get_all_entities_from_taxonomy_tree(
    first_level_realmittel_taxonomies
)


display(
    Markdown(
        f"""
### Ergebnis
{len(all_gymnasium_entities)} Inhalte sind für Gymnasium geeignet.  
{len(all_realmittel_entities)} Inhalte sind für Real-, Mittel- und Gesamtschule geeignet.
"""
    )
)
# TODO: Prüfungen weren't systematically added.


### Ergebnis
2670 Inhalte sind für Gymnasium geeignet.  
3936 Inhalte sind für Real-, Mittel- und Gesamtschule geeignet.


In [10]:
from collections import Counter

repeated = list((Counter(all_realmittel_entities) & Counter(all_gymnasium_entities)).elements())

display(
    Markdown(
        f"""
{len(repeated)} Inhalte sind für alle Schultypen geeignet.  
"""
    )
)


890 Inhalte sind für alle Schultypen geeignet.  
